In [1]:
import albumentations as A
import torchvision.transforms as tt
from PIL import Image
import cv2
import os
from pathlib import Path

In [2]:
transform = A.Compose([A.Resize(width=240,height=320),
    A.RandomCrop(width=240, height=320),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.9,brightness_limit = 0.15),A.GaussNoise(p=0.8),
A.Rotate(limit=25,p=1)])

In [3]:
def getImagePaths(path,str_format = True):
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            fullpath = os.path.join(dirname, filename)
            image_names.append(fullpath)
    if str_format:
        return image_names
    else:
        return [Path(i) for i in image_names]

In [4]:
def image_augmentation(img,mask,trf): 
    transformed = trf(image=img, mask=mask)
    transformed_image = transformed['image']
    transformed_mask = transformed['mask']
    
    return transformed_image, transformed_mask

In [5]:
def full_augm(default_path,default_mask_path,path_to_save,path_to_save_mask,num_augs):
    try:
        os.makedirs(path_to_save)
        os.makedirs(path_to_save_mask)
    except:
        pass
    vice_transformation = tt.ToPILImage()
    pics = getImagePaths(default_path,str_format=False)
    masks = getImagePaths(default_mask_path,str_format=False)
    for pic_path,mask_path in zip(pics,masks):
        pic_num = pic_path.name.split('.')[0]
        image = cv2.imread(str(pic_path))
        mask = cv2.imread(str(mask_path))
        for j in range(num_augs):
            tr_img,tr_m = image_augmentation(image,mask,transform)
            tr_m = tr_m[:,:,0]
            #tr_m = cv2.cvtColor(tr_m, cv2.COLOR_GR)
            tr_img = cv2.cvtColor(tr_img, cv2.COLOR_BGR2RGB)
            tr_img = Image.fromarray(tr_img)
            tr_m = Image.fromarray(tr_m)
            
            tr_img.save(path_to_save+'/'+str(pic_num)+'_'+str(j)+'.jpg')
            tr_m.save(path_to_save_mask+'/'+str(pic_num)+'_'+str(j)+'.jpg')
    
    

In [6]:
full_augm('data/train/','data/train_mask/','data/augm_tr','data/augm_tr_mask',3)

In [7]:
full_augm('../MINDSET/human_dataset/data/Training_Images/','../MINDSET/human_dataset/data/Ground_Truth/','data/augm_extra_tr',
         'data/augm_extra_tr_mask',3)